# Exploration using Tidymodels

# The Surfing for Science data base

Load the `tidymodels` library along with other useful ones:

In [ ]:
library(tidymodels)

── Attaching packages ────────────────────────────────────── tidymodels 1.3.0 ──

✔ broom        1.0.8     ✔ recipes      1.3.0
✔ dials        1.4.0     ✔ rsample      1.3.0
✔ dplyr        1.1.4     ✔ tibble       3.2.1
✔ ggplot2      3.5.2     ✔ tidyr        1.3.1
✔ infer        1.0.8     ✔ tune         1.3.0
✔ modeldata    1.4.0     ✔ workflows    1.2.0
✔ parsnip      1.3.1     ✔ workflowsets 1.1.0
✔ purrr        1.0.4     ✔ yardstick    1.3.2

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ purrr::discard() masks scales::discard()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()
✖ recipes::step()  masks stats::step()


Attaching package: 'readr'

The following object is masked from 'package:yardstick':

    spec

The following object is masked from 'package:scales':

    col_factor

Load the full dataset from the Surging for Science project, also clean the variables names so they use `snake_case` and skip superfluous metadata. Character predictors are converter to type `factor` for compatibility with recipes.

In [ ]:
plastics <- read_csv("data/surfingforscience_240325.csv",
                     col_types = cols(
                       Particle_Num = col_skip(), 
                       Cruise_Name = col_skip(), Transect = col_skip(), 
                       Replicate = col_skip(), Sieve = col_skip(), 
                       Subsample = col_skip(), File_Name = col_skip(), 
                       Modified = col_skip(), ImageType = col_skip())) |> 
janitor::clean_names() |> 
mutate_if(is.character, as.factor)

The aim of this notebook is to evaluate performance of different models following the [tidymodels](https://www.tidymodels.org/) workflow. To evaluate performance we need plastics that have been evaluated manually so we can compare the trained human observer with the AI. A *good* model is the one that gets us very similar results to the trained human.

Subset the data to get all plastics evaluated by a human observer.

In [ ]:
plastics_manual <- plastics |> filter(rf_use == FALSE)

Overall, the model currently in use had a 58.9% success rate.

In [ ]:
plastics_manual |> 
  count(rf_success = as.character(group) == as.character(rf_group)) |> 
  mutate(prop = n/sum(n))

# A tibble: 2 × 3
  rf_success     n  prop
  <lgl>      <int> <dbl>
1 FALSE      23656 0.411
2 TRUE       33857 0.589

Remove all variables related to the current model

In [ ]:
plastics_manual <- plastics_manual |> 
  select(-starts_with("prob"), -starts_with("rf"))

Before fitting new models, data splitting is done to avoid training and evaluating the model with the same set of data. If so was done, the performance results would be overly optimistic due to overfitting. The training dataset is intended to have plastics of all types in similar proportion, this is achieved by setting the argument `strata = group`.

In [ ]:
plastics_split <- plastics_manual |> initial_split(prop = 3/4, strata = group)

After checking the relative group composition of our training set we observe that some groups are very rare: `Fibre.bundle`, `Paint.chip` and `NA.Other`

In [ ]:
train_plastics <- training(plastics_split)
test_plastics <- testing(plastics_split)

train_plastics |> count(group) |> mutate(prop = n/sum(n))

# A tibble: 11 × 3
   group                  n      prop
   <fct>              <int>     <dbl>
 1 Artificial.turf      847 0.0196   
 2 Fibre                107 0.00248  
 3 Fibre.bundle           7 0.000162 
 4 Filament             845 0.0196   
 5 Film.Sheet         11088 0.257    
 6 Foam                3602 0.0835   
 7 Fragment           26176 0.607    
 8 NA.Other              37 0.000858 
 9 Paint.chip             1 0.0000232
10 Pellet               142 0.00329  
11 Spherule.Microbead   282 0.00654  

# Preprocessin using recipes

Let’s initialize a simple recipe for our first model

In [ ]:
plastics_recipe <- train_plastics |> recipe(group ~ .) |> 
  step_dummy(all_nominal_predictors()) |> 
  step_zv(all_predictors())

Next, define a random forest model of 1000 trees using the `ranger` engine.

In [ ]:
rf_mod <- 
  rand_forest(trees = 100) |> 
  set_engine("ranger") |> 
  set_mode("classification")

Create a modelling workflow that combines the model and the recipe

In [ ]:
plastics_wflow <- workflow() |> add_model(rf_mod) |> add_recipe(plastics_recipe)

Fit the workflow to the training data

> **Warning**
>
> Heavy computation ahead

In [ ]:
rf_fit <- plastics_wflow |> fit(data = train_plastics)

Let’s see how it went

In [ ]:
rf_fit |> extract_fit_parsnip()

parsnip model object

Ranger result

Call:
 ranger::ranger(x = maybe_data_frame(x), y = y, num.trees = ~100,      num.threads = 1, verbose = FALSE, seed = sample.int(10^5,          1), probability = TRUE) 

Type:                             Probability estimation 
Number of trees:                  100 
Sample size:                      43134 
Number of independent variables:  152 
Mtry:                             12 
Target node size:                 10 
Variable importance mode:         none 
Splitrule:                        gini 
OOB prediction error (Brier s.):  0.2001236 

# Evaluating performance using resamples

Lets create a 10 fold cross validation resample

In [ ]:
set.seed(567)
folds <- train_plastics |> vfold_cv(v = 4)

Now fit the workflow to the resamples

> **Warning**
>
> Heavy computation ahead

In [ ]:
set.seed(311)
rf_fit_rs <- plastics_wflow |> fit_resamples(folds)

→ A | warning: Dropped unused factor level(s) in dependent variable: Paint.chip.

There were issues with some computations   A: x1

→ B | error:   Failed to compute `roc_auc()`.
               Caused by error:
               ! Can't select columns that don't exist.
               ✖ Column `.pred_Paint.chip` doesn't exist.

There were issues with some computations   A: x1
There were issues with some computations   A: x1   B: x1
→ C | warning: ✖ No observations were detected in `truth` for level: Paint.chip.
               ℹ Computation will proceed by ignoring those levels.
There were issues with some computations   A: x1   B: x1
There were issues with some computations   A: x1   B: x1   C: x1
There were issues with some computations   A: x1   B: x1   C: x2
There were issues with some computations   A: x1   B: x1   C: x3
There were issues with some computations   A: x1   B: x1   C: x3

Finally, collect the metrics and see how it did

In [ ]:
rf_fit_rs |> collect_metrics()

# A tibble: 3 × 6
  .metric     .estimator  mean     n  std_err .config             
  <chr>       <chr>      <dbl> <int>    <dbl> <chr>               
1 accuracy    multiclass 0.762     3 0.00227  Preprocessor1_Model1
2 brier_class multiclass 0.172     3 0.000966 Preprocessor1_Model1
3 roc_auc     hand_till  0.863     3 0.00977  Preprocessor1_Model1